🌸 The Plan

1- Extract CV Text ✅

2- Compare the extracted text with the Job Describtion **(Agent 1)** ✅

3- IF Accepted --> Generate Questions **(Agent 2)** ⭐: --> Send acceptance email ✅

4- IF Rejected --> Recommend CV imporvement or recommend some courses ✅ **(Agent 3)** --> Send rejection email ✅

In [162]:
# !pip install pdfplumber python-docx pypdf docx2txt
# !pip install langchain langchain-openai langchain-classic langchain_community langfuse

# Import Libraries

In [163]:
import os
import re
import pypdf
from docx import Document
import json
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langgraph.graph import StateGraph, END
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List, TypedDict, Optional

from dotenv import dotenv_values
from google.colab import userdata

import smtplib
from email.message import EmailMessage

from langfuse.langchain import CallbackHandler

# OpenAI Model

**Langfuse Keys**

In [164]:
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"
langfuse_handler = CallbackHandler()

In [165]:
# API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# # without Langfuse
# # Identify the used model
# llm = ChatOpenAI(
#     model="gpt-4.1",
#     temperature=0
# )

llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0,
    callbacks=[langfuse_handler]
)

# Read and Extract CV Text

**Extract CV Text using PDF Loader and Docx Loader (Docx Loader is not working Perfectly)**

In [166]:
# def extract_cv_text(cv_path):

#     if not os.path.exists(cv_path):
#         raise FileNotFoundError(f"File not found: {cv_path}")

#     documents = []

#     # PDF
#     if cv_path.lower().endswith(".pdf"):
#         loader = PyPDFLoader(cv_path)
#         documents = loader.load()

#     # DOCX
#     elif cv_path.lower().endswith(".docx"):
#         loader = Docx2txtLoader(cv_path)
#         documents = loader.load()

#     else:
#         raise ValueError("Unsupported file type. Use PDF or DOCX.")

#     # Combine all pages into one text
#     full_text = "\n".join(doc.page_content for doc in documents)

#     text_lines = full_text.splitlines()

#     # Extract Gmail address
#     gmail_match = re.search(r"[a-zA-Z0-9._%+-]+@gmail\.com", full_text)
#     gmail = gmail_match.group(0) if gmail_match else None

#     # Extract candidate name (first reasonable line)
#     name = None
#     for line in text_lines:
#         clean_line = line.strip()
#         if clean_line and not re.search(r"@|[0-9]{2,}", clean_line):
#             name = clean_line
#             break

#     return name, gmail, full_text

**Extract CV text using PDF Loader and python-docx (Better Results)**

In [167]:
def extract_cv_text(cv_path):
    if not os.path.exists(cv_path):
        raise FileNotFoundError(f"File not found: {cv_path}")

    full_text = ""

    # PDF
    if cv_path.lower().endswith(".pdf"):
        loader = PyPDFLoader(cv_path)
        documents = loader.load()
        full_text = "\n".join(doc.page_content for doc in documents)

    # DOCX
    elif cv_path.lower().endswith(".docx"):
        # Using python-docx
        doc = Document(cv_path)
        paragraphs = []
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():  # Skip empty paragraphs
                paragraphs.append(paragraph.text)

        # Also extract text from tables
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                        if paragraph.text.strip():
                            paragraphs.append(paragraph.text)

        full_text = "\n".join(paragraphs)

    else:
        raise ValueError("Unsupported file type. Use PDF or DOCX.")

    text_lines = full_text.splitlines()

    # Extract Gmail address
    gmail_match = re.search(r"[a-zA-Z0-9._%+-]+@gmail\.com", full_text)
    gmail = gmail_match.group(0) if gmail_match else None

    # Extract candidate name (first reasonable line)
    name = None
    for line in text_lines:
        clean_line = line.strip()
        if clean_line and not re.search(r"@|[0-9]{2,}", clean_line):
            name = clean_line
            break

    return name, gmail, full_text

In [168]:
# name, email, cv_text = extract_cv_text("e.docx")
# print(name)
# print(email)
# print(cv_text)

# Output Schema for The CV Screening Results

In [169]:
class CVMatchResult(BaseModel):
    decision: str = Field(description="ACCEPT or REJECT")
    match_score: int = Field(description="Score from 0 to 100")
    strengths: List[str]
    missing_skills: List[str]
    internal_reasoning: str = Field(
        description="Internal HR evaluation written in third person."
    )
    candidate_reasoning: str = Field(
        description=(
            "Candidate-facing explanation written directly to the candidate "
            "using 'you' and 'your'. Professional, respectful, and encouraging. "
            "Do NOT use third-person phrases like 'the candidate'."
        )
    )

# Output Schema for The CV Improvement

In [170]:
class ImprovementFeedback(BaseModel):
    recommended_courses: List[str] = Field(
        description="Courses for missing skills"
    )
    spelling_issues: List[str] = Field(
        description="Spelling or grammar issues detected"
    )
    arrangement_issues: List[str] = Field(
        description="Section ordering or grouping issues"
    )
    consistency_issues: List[str] = Field(
        description="Inconsistent formatting (dates, capitalization, bullets)"
    )
    readability_issues: List[str] = Field(
        description="Long paragraphs or unclear structure issues"
    )

# Output Schema for The Technical Questions

In [171]:
class TechnicalInterviewQuestions(BaseModel):
    questions: List[str] = Field(
        description=(
            "Five technical interview questions tailored to the candidate's CV. "
            "Questions must be realistic, commonly asked in real interviews, "
            "and strictly based on skills, tools, projects, or experience explicitly "
            "mentioned in the CV."
        )
    )


# Screening Prompt  , very important

In [172]:
screening_prompt = ChatPromptTemplate.from_template("""
You are an expert HR recruiter responsible for screening candidates.

Your task is to evaluate how well the candidate's CV matches the Job Description
and decide whether the candidate should be ACCEPTED or REJECTED.

Evaluation Guidelines:
- Focus on:
  • Technical skills and technologies explicitly mentioned
  • Years of experience and relevance to the role
  • Education, certifications, and academic background
  • Project and practical experience related to the job
- Ignore formatting, grammar, and layout issues
- Do NOT assume or hallucinate skills not clearly stated in the CV
- Do NOT penalize junior candidates for lack of years if the role allows entry-level profiles

Scoring Rules:
- 90–100: Excellent match
- 75–89: Strong match
- 60–74: Partial match
- Below 60: Weak match

Decision Rules:
- ACCEPT if match_score ≥ 75
- REJECT if match_score < 75

Missing Skills:
- Include important skills or requirements explicitly mentioned in the Job Description

Reasoning Instructions:
- Provide TWO versions of reasoning:

  1. internal_reasoning:
     - Written in third person (e.g., "the candidate")
     - Objective and analytical

  2. candidate_reasoning:
     - Written directly to the candidate using "you" and "your"
     - Professional, respectful, and encouraging
     - Explain strengths first, then gaps
     - Do NOT use third-person phrases such as "the candidate"

{format_instructions}

CV:
{candidate_cv}

Job Description:
{job_description}
""")

# Output of the LLM must match the parser
screening_parser = JsonOutputParser(pydantic_object=CVMatchResult)

# Chain
screening_chain = (screening_prompt.partial(
    format_instructions=screening_parser.get_format_instructions()
    )
    | llm
    | screening_parser
)

# Improvement Prompt

In [173]:
improvement_prompt = ChatPromptTemplate.from_template("""
You are an expert CV reviewer and technical career advisor.

Your task is to provide Learning Recommendations ONLY.

Rules:
- Recommend courses ONLY for missing skills
- Use trusted platforms only:
  • Coursera
  • Udemy
  • edX
  • DataCamp
  • Any other reliable resources you find online
- Do NOT invent certifications
- Do NOT comment on CV writing, formatting, or structure
- Do NOT suggest new skills beyond the provided missing skills

Missing Skills:
{missing_skills}

Return valid JSON only.

{format_instructions}
""")

# Output of the LLM must match the parser
feedback_parser = JsonOutputParser(pydantic_object=ImprovementFeedback)

# Chain
feedback_chain = (
    improvement_prompt.partial(
        format_instructions=feedback_parser.get_format_instructions()
    )
    | llm
    | feedback_parser
)

# Questions Prompt الفرق بينه و بين جزئيتى؟

In [174]:
interview_questions_prompt = ChatPromptTemplate.from_template("""
You are a senior technical interviewer conducting a real-world interview.

Your task:
Generate EXACTLY 5 technical interview questions.

IMPORTANT CONTEXT — Strengths:
- "Strengths" represent the **validated, matched skills and competencies** between the CV and the job description
- They were produced by a prior screening step and should be treated as **high-confidence areas**
- Strengths MUST guide topic selection and prioritization
- Do NOT mention strengths explicitly in the questions
- Do NOT invent new strengths beyond what is provided
{Strengths}

⚠️ Question composition is STRICT and MUST follow this order:

1️⃣ Question 1 — General Field Technique:
- Ask about ONE widely used technique, method, or concept in the candidate’s field
- It must be relevant to the job description
- It must NOT be tied to a specific project in the CV
- Example (AI): anomaly detection, model evaluation strategies, handling data imbalance
- Keep it practical and experience-oriented, not textbook

2️⃣ Questions 2 & 3 — CV-Based Techniques (ABSTRACTED):

- Each question must focus on a technique, method, or approach explicitly mentioned in the CV
- The technique must be discussed **independently of any specific project or use case**
- You may reference example model families (e.g., transformers, tree-based models), but:
  • Do NOT tie the question to a particular application domain
  • Do NOT frame the question around a specific project outcome
- If project names were removed from the CV, the question should still make sense
- Focus on:
  • When and why the technique is used
  • Trade-offs compared to alternatives
  • Practical constraints or limitations
- Avoid phrasing that encourages project storytelling


3️⃣ Questions 4 & 5 — Project-Focused (High-Level):
- Each question must relate to a project mentioned in the CV
- Questions should focus on:
  • System or solution design
  • Key technical decisions
  • Problem-solving approach
- Do NOT ask about tiny implementation details
- Do NOT ask for full project walkthroughs
- Keep the scope broad enough to allow multiple valid answers

How the questions should sound:
- Natural, conversational, and human
- Concise (1–2 sentences max per question)
- Open-ended (cannot be answered with yes/no)
- Designed to be answered in **2 minutes or less**
- Practical and experience-focused
- Slightly probing, but not academic

Difficulty and scope rules:
- Questions must sit in the middle ground:
  • Not generic interview questions
  • Not overly specific to one CV line
- Avoid niche theory, formulas, or academic framing

Evaluation-awareness (IMPORTANT):
- Each question must have a clear intent and evaluation signal
- Another agent should be able to assess:
  • Whether the answer directly addresses the question
  • Depth vs. vagueness
  • Practical understanding vs. surface-level theory

What to avoid:
- Long explanations inside the question
- Robotic or overly formal language
- “Walk me through your entire project” phrasing
- Repeating CV text verbatim
- Inventing skills, tools, or experience

CV:
{candidate_cv}

Job Description:
{job_description}

Return valid JSON only.

{format_instructions}
""")

interview_questions_parser = JsonOutputParser(
    pydantic_object=TechnicalInterviewQuestions
)

interview_questions_chain = (
    interview_questions_prompt.partial(
        format_instructions=interview_questions_parser.get_format_instructions()
    )
    | llm
    | interview_questions_parser
)


In [ ]:
screening_prompt = ChatPromptTemplate.from_template("""
You are an expert HR recruiter responsible for screening candidates.

Your task is to evaluate how well the candidate's CV matches the Job Description
and decide whether the candidate should be ACCEPTED or REJECTED.

Evaluation Guidelines:
- Focus on:
  • Technical skills and technologies explicitly mentioned
  • Years of experience and relevance to the role
  • Education, certifications, and academic background
  • Project and practical experience related to the job
- Ignore formatting, grammar, and layout issues
- Do NOT assume or hallucinate skills not clearly stated in the CV
- Do NOT penalize junior candidates for lack of years if the role allows entry-level profiles

Scoring Rules:
- 90–100: Excellent match
- 75–89: Strong match
- 60–74: Partial match
- Below 60: Weak match

Decision Rules:
- ACCEPT if match_score ≥ 75
- REJECT if match_score < 75

Missing Skills:
- Include important skills or requirements explicitly mentioned in the Job Description

Reasoning Instructions:
- Provide TWO versions of reasoning:

  1. internal_reasoning:
     - Written in third person (e.g., "the candidate")
     - Objective and analytical

  2. candidate_reasoning:
     - Written directly to the candidate using "you" and "your"
     - Professional, respectful, and encouraging
     - Explain strengths first, then gaps
     - Do NOT use third-person phrases such as "the candidate"

{format_instructions}

CV:
{candidate_cv}

Job Description:
{job_description}
""")

# Output of the LLM must match the parser
screening_parser = JsonOutputParser(pydantic_object=CVMatchResult)

# Chain
screening_chain = (screening_prompt.partial(
    format_instructions=screening_parser.get_format_instructions()
    )
    | llm
    | screening_parser
)

# Acceptance and Rejection Email Templates

In [175]:
ACCEPTANCE_EMAIL = """
Dear {candidate_name},

Thank you for taking the time to apply for the {position} position.

We are pleased to inform you that, after reviewing your profile, your skills and experience align well with our requirements. Based on this evaluation, we would like to move forward with your application to the next stage of the hiring process.

Our team will contact you shortly with further details.

Thank you for your interest, and we look forward to speaking with you soon.

Best regards,
HR Team
"""

REJECTION_EMAIL = """
Dear {candidate_name},

Thank you for your interest in the {position} position and for taking the time to apply.

After careful consideration, we regret to inform you that we will not be moving forward with your application at this time.

{reasoning}

This decision does not reflect a lack of potential, and we encourage you to continue developing your skills and to apply again in the future.

We appreciate your interest and wish you the best in your job search.

Kind regards,
HR Team
"""

# Job description

In [176]:
selected_position = "Data Engineer"
company = "Bla Bla"

In [177]:
JOB_DESCRIPTIONS_PATH = Path("job_descriptions.json")

with open(JOB_DESCRIPTIONS_PATH, "r", encoding="utf-8") as f:
    JOB_DESCRIPTIONS = json.load(f)

# JOB_DESCRIPTIONS

In [178]:
if selected_position not in JOB_DESCRIPTIONS:
    raise ValueError(
        f"Position '{selected_position}' not found. "
        f"Available positions: {list(JOB_DESCRIPTIONS.keys())}"
    )

job_description = JOB_DESCRIPTIONS[selected_position]["job_description"]

# Shared States

In [179]:
class ATSState(TypedDict):
    cv_path: str
    candidate_name: Optional[str]
    candidate_email: Optional[str]
    candidate_cv: Optional[str]

    screening_result: Optional[dict]
    improvement_feedback: Optional[dict]
    interview_questions: Optional[dict]   # 👈 NEW

# Sender Email Info

In [180]:
SENDER_EMAIL = userdata.get("SENDER_EMAIL")
SENDER_PASSWORD = userdata.get("SENDER_PASSWORD")

# The Graph Nodes

**Extract CV text node**

In [181]:
def extract_cv_node(state: ATSState) -> ATSState:
    print("▶ Extracting CV text")
    name, email, cv_text = extract_cv_text(state["cv_path"])

    print(f"  ✓ Extracted name: {name}")
    print(f"  ✓ Extracted email: {email}")

    return {
        **state,
        "candidate_name": name,
        "candidate_email": email,
        "candidate_cv": cv_text,
    }

In [182]:
def screening_node(state: ATSState) -> ATSState:
    print("▶ Evaluating CV vs job description")

    # # Without Langfuse
    # result = screening_chain.invoke({
    #     "candidate_cv": state["candidate_cv"],
    #     "job_description": job_description
    # })


    # With Langfuse
    result = screening_chain.invoke(
      {
        "candidate_cv": state["candidate_cv"],
        "job_description": job_description
      },
    config={
        "run_name": "screening_cv_vs_jd"
      }
    )


    print(f"  ✓ Decision: {result['decision']}")
    print(f"  ✓ Match score: {result['match_score']}")

    return {
        **state,
        "screening_result": result
    }

In [183]:
def decision_router(state: ATSState) -> str:
    if state["screening_result"]["decision"] == "ACCEPT":
        return "accept"
    return "reject"

In [184]:
def improvement_node(state: ATSState) -> ATSState:
    print("▶ Generating CV improvement feedback")


    # # Without Langfuse
    # feedback = feedback_chain.invoke({
    #     "missing_skills": state["screening_result"]["missing_skills"],
    #     "candidate_cv": state["candidate_cv"]
    # })



    # With Langfuse
    feedback = feedback_chain.invoke(
      {
        "missing_skills": state["screening_result"]["missing_skills"]
      },
    config={
        "run_name": "cv_improvement_feedback"
      }
    )

    print("Improvement feedback generated ✅")

    return {
        **state,
        "improvement_feedback": feedback
    }

In [185]:
def send_acceptance_email_node(state: ATSState) -> ATSState:
    print("▶ Send Acceptance Email")

    if not state["candidate_email"]:
        print("No candidate email found. Skipping acceptance email.")
        return state

    email_body = ACCEPTANCE_EMAIL.format(
        candidate_name=state["candidate_name"] or "Candidate",
        position=selected_position
    )

    msg = EmailMessage()
    msg["Subject"] = f"Application Update – {company}"
    msg["From"] = SENDER_EMAIL
    msg["To"] = state["candidate_email"]
    msg.set_content(email_body)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
            print("Acceptance email sent successfully ✅")
    except Exception as e:
        print(f"Error sending acceptance email: {e}")

    return state

In [186]:
def send_rejection_email_node(state: ATSState) -> ATSState:
    print("▶ Send Rejection Email")
    if not state["candidate_email"]:
        print("No candidate email found. Skipping rejection email.")
        return state

    reasoning_text = state["screening_result"]["candidate_reasoning"]

    feedback = state.get("improvement_feedback")

    if feedback:
        sections = []

        if feedback.get("recommended_courses"):
            sections.append(
                "Recommended learning resources:\n"
                + "\n".join(f"- {c}" for c in feedback["recommended_courses"])
            )

        if feedback.get("spelling_issues"):
            sections.append(
                "Spelling or grammar issues to review:\n"
                + "\n".join(f"- {i}" for i in feedback["spelling_issues"])
            )

        if feedback.get("arrangement_issues"):
            sections.append(
                "CV section organization suggestions:\n"
                + "\n".join(f"- {i}" for i in feedback["arrangement_issues"])
            )

        if feedback.get("consistency_issues"):
            sections.append(
                "Formatting consistency notes:\n"
                + "\n".join(f"- {i}" for i in feedback["consistency_issues"])
            )

        if feedback.get("readability_issues"):
            sections.append(
                "Readability improvements:\n"
                + "\n".join(f"- {i}" for i in feedback["readability_issues"])
            )

        if sections:
            reasoning_text += "\n\n" + "\n\n".join(sections)

    email_body = REJECTION_EMAIL.format(
        candidate_name=state["candidate_name"] or "Candidate",
        position=selected_position,
        reasoning=reasoning_text
    )

    msg = EmailMessage()
    msg["Subject"] = f"Application Update – {company}"
    msg["From"] = SENDER_EMAIL
    msg["To"] = state["candidate_email"]
    msg.set_content(email_body)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
            print("Rejection email sent successfully ✅")
    except Exception as e:
        print(f"Error sending rejection email: {e}")

    return state

In [187]:
def interview_questions_node(state: ATSState) -> ATSState:
    print("▶ Generating technical interview questions")

    questions = interview_questions_chain.invoke(
        {
            "candidate_cv": state["candidate_cv"],
            "job_description": job_description,
            "Strengths": state["screening_result"]["strengths"] # Changed 'strengths' to 'Strengths'
        },
        config={
            "run_name": "technical_interview_questions"
        }
    )

    print("\n📌 Technical Interview Questions:")
    for i, q in enumerate(questions["questions"], start=1):
        print(f"{i}. {q}")

    return {
        **state,
        "interview_questions": questions
    }

# Build The Graph

In [188]:
# graph = StateGraph(ATSState)

# # Register nodes
# graph.add_node("extract_cv", extract_cv_node)
# graph.add_node("screening", screening_node)
# graph.add_node("improvement", improvement_node)
# graph.add_node("send_accept", send_acceptance_email_node)
# graph.add_node("send_reject", send_rejection_email_node)


# # Entry point
# graph.set_entry_point("extract_cv")

# # Main flow
# graph.add_edge("extract_cv", "screening")

# graph.add_conditional_edges(
#     "screening",
#     decision_router,
#     {
#         "accept": "send_accept",
#         "reject": "improvement"
#     }
# )

# graph.add_edge("improvement", "send_reject")

# graph.add_edge("send_accept", END)
# graph.add_edge("send_reject", END)

In [189]:
graph = StateGraph(ATSState)

# Register nodes
graph.add_node("extract_cv", extract_cv_node)
graph.add_node("screening", screening_node)
graph.add_node("improvement", improvement_node)
graph.add_node("interview_questions", interview_questions_node)  # ✅ NEW
graph.add_node("send_accept", send_acceptance_email_node)
graph.add_node("send_reject", send_rejection_email_node)

# Entry point
graph.set_entry_point("extract_cv")

# Main flow
graph.add_edge("extract_cv", "screening")

graph.add_conditional_edges(
    "screening",
    decision_router,
    {
        "accept": "interview_questions",  # ✅ ACCEPT → generate questions first
        "reject": "improvement"
    }
)

# Rejection path
graph.add_edge("improvement", "send_reject")


# Acceptance path
graph.add_edge("interview_questions", "send_accept")  # ✅ NEW
graph.add_edge("send_accept", END)
graph.add_edge("send_reject", END)

# Execute The Graph

In [191]:
ats_graph = graph.compile()

# final_state = ats_graph.invoke({
#     "cv_path": "autoCV__2_.pdf"
# })
final_state = ats_graph.invoke(
    {"cv_path": "e.docx"},
    config={"run_name": "ATS_full_run"}
)

▶ Extracting CV text
  ✓ Extracted name: Eman Mahmoud Abdel-Halim Muhammad
  ✓ Extracted email: None
▶ Evaluating CV vs job description
  ✓ Decision: ACCEPT
  ✓ Match score: 85
▶ Generating technical interview questions

📌 Technical Interview Questions:
1. Can you describe your approach to designing an efficient ETL pipeline for processing large volumes of data, and what factors you consider to ensure reliability and scalability?
2. What are some best practices you follow when writing Python code for data transformation tasks, and how do you handle performance bottlenecks?
3. When working with SQL and data warehouses, how do you approach data modeling to balance query performance and maintainability?
4. In your project building a data warehouse and interactive dashboard for a government or industry client, what were the key technical decisions you made regarding data integration and visualization?
5. Can you walk me through how you approached automating the manual data cleansing proces

In [ ]:
# print("LANGFUSE_PUBLIC_KEY:", os.environ.get("LANGFUSE_PUBLIC_KEY"))
# print("LANGFUSE_SECRET_KEY:", os.environ.get("LANGFUSE_SECRET_KEY"))
# print("LANGFUSE_HOST:", os.environ.get("LANGFUSE_HOST"))


In [ ]:
ats_graph